In [1]:
import pandas as pd
import numpy as np
import time
import unicodedata
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, WebDriverException

import csv
import os

import sys
sys.path.append('./lib/')
from functions import *
from tqdm import tnrange, tqdm_notebook
from unidecode import unidecode

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import lxml.html
import lxml

import glob

In [2]:
files = glob.glob('output/**/*.csv', recursive=True)
files
    

['output\\debugging.csv',
 'output\\log_error.csv',
 'output\\log_error_20170215.csv',
 'output\\log_error_20170221.csv',
 'output\\log_error_20170224.csv',
 'output\\log_error_20170225-1.csv',
 'output\\log_error_20170225.csv',
 'output\\log_opened.csv',
 'output\\log_opened_20170212.csv',
 'output\\log_opened_20170214.csv',
 'output\\log_opened_20170215.csv',
 'output\\log_opened_20170221.csv',
 'output\\log_opened_20170224.csv',
 'output\\log_opened_20170225-1.csv',
 'output\\log_opened_20170225.csv',
 'output\\retake_entity_0.csv',
 'output\\retake_entity_1.csv',
 'output\\scraped_data.csv',
 'output\\scraped_data_20170212.csv',
 'output\\scraped_data_20170214.csv',
 'output\\scraped_data_20170215-1.csv',
 'output\\scraped_data_20170215.csv',
 'output\\scraped_data_20170216.csv',
 'output\\scraped_data_20170221.csv',
 'output\\scraped_data_20170224.csv',
 'output\\scraped_data_20170225.csv',
 'output\\scraped_data_clean.csv',
 'output\\output_1\\log_error_1.csv',
 'output\\output_1

In [3]:
col_names = ['entity', 'department', 'contract', 'year', 'month', 'group', 'lastn', 'lastn2', 'givenn', 'degree', 
             'qualif', 'position', 'region', 'specials', 'currency', 'salary', 'overt', 'start_date', 'end_date', 'obs', 'url', 'other']
len(col_names)

22

In [4]:

dfscraped = pd.DataFrame()

for f in files:
    if( ('scraped' in f) and (not 'data_clean' in f)):
        df = pd.read_csv(f, header=None, encoding='latin1', names=col_names, warn_bad_lines=True, low_memory=False)
        df['filename'] = f
        dfscraped = pd.concat([dfscraped, df], ignore_index=True)

print(dfscraped.shape)

(1899135, 23)


In [5]:
dfscraped.filename.value_counts()

output\scraped_data_20170225.csv                 998922
output\output_1\scraped_data_1.csv               360598
output\output_1\scraped_data_20170224_1.csv      193449
output\scraped_data_20170224.csv                  78909
output\output_1\scraped_data_20170223_1.csv       67889
output\scraped_data_20170215.csv                  50586
output\scraped_data_20170214.csv                  44783
output\scraped_data_20170216.csv                  36591
output\scraped_data_20170212.csv                  25227
output\scraped_data_20170221.csv                  15300
output\output_1\old\scraped_data_2_backup.csv      9215
output\output_1\old\scraped_data.csv               8224
output\scraped_data.csv                            4531
output\scraped_data_20170215-1.csv                 3145
output\output_1\old\scraped_data_old.csv           1766
Name: filename, dtype: int64

In [6]:
dfscraped.entity.value_counts()

Ministerio de Justicia                              336990
Ministerio de Vivienda y Urbanismo                  196956
Ministerio de Obras Públicas                        183468
Ministerio de Educación                             128751
Ministerio del Trabajo y Previsión Social           113612
entity                                               92508
Ministerio de Economía, Fomento y Turismo            92159
Ministerio de Desarrollo Social                      84779
Ministerio del Trabajo y PrevisiÃ³n Social           80198
Ministerio de Defensa Nacional                       63995
Consejo de Defensa del Estado                        52169
b'Ministerio del Interior y Seguridad Pblica'        47911
Ministerio de Agricultura                            44494
b'Ministerio del Trabajo y Previsin Social'          43390
Ministerio del Interior y Seguridad Pública          42636
Ministerio de MinerÃ­a                               37899
b'Ministerio de Obras Pblicas'                       278

In [7]:
dfscraped.url.nunique()

20720

## Data that was not captured with the right encoding, get again

In [8]:
retake_entity = list(dfscraped[dfscraped.entity.str.contains("b'", na=False)].entity.value_counts().index)
retake_entity = retake_entity + list(dfscraped[dfscraped.entity.str.contains("Ã", na=False)].entity.value_counts().index)
retake_entity = retake_entity + list(dfscraped[dfscraped.entity.str.contains("Ã­", na=False)].entity.value_counts().index)
retake_entity = retake_entity + ['Ministerio del Interior y Seguridad Pblica', 'Presidencia de la Repblica ']  

In [9]:
print('Total Urls', dfscraped.url.nunique())
print('Urls to retake', dfscraped[dfscraped.entity.isin(retake_entity)].url.nunique())

Total Urls 20720
Urls to retake 3918


In [10]:
retake_entity_urls = []
for u in list(set(dfscraped.ix[dfscraped.entity.isin(retake_entity), 'url'].unique())):
    if str(u) != 'nan':
        if "b'" in u:
            u = u[2:]
        if u[-1] == "'":
            u = u[:len(u)-1]
        retake_entity_urls.append(u)
len(retake_entity_urls)

3918

In [11]:
retake_entity_urls

['http://www.gobiernotransparentechile.cl/directorio/entidad/2/8/per_contrata/2013/Noviembre',
 'http://www.gobiernotransparentechile.cl/directorio/entidad/8/136/per_contrata/Ao-2015',
 'http://www.gobiernotransparentechile.cl/directorio/entidad/25/351/per_planta/A%C3%B1o%202016/Marzo',
 'http://www.gobiernotransparentechile.cl/directorio/entidad/12/173/per_contrata/A%EF%BF%BDo%202011/Marzo?x=0&y=0&page_number=4&sort=id&direction=asc',
 'http://www.gobiernotransparentechile.cl/directorio/entidad/10/146/per_contrata/Ao-2014?x=0&y=0&page_number=47&sort=id&direction=asc',
 'http://www.gobiernotransparentechile.cl/directorio/entidad/12/173/per_contrata/A%EF%BF%BDo%202014/Septiembre',
 'http://www.gobiernotransparentechile.cl/directorio/entidad/12/169/per_contrata/Ao-2015?x=0&y=0&page_number=11&sort=id&direction=asc',
 'http://www.gobiernotransparentechile.cl/directorio/entidad/13/283/per_planta/A%EF%BF%BDo%202016/Mayo',
 'http://www.gobiernotransparentechile.cl/directorio/entidad/8/136/per

### Open browser and re-scrape data 

In [17]:
output_file = output_file = './output/retake_entity_0.csv'

In [18]:
if not os.path.isfile(output_file):
    browser = webdriver.Firefox()
    for url in retake_entity_urls[0:5]:
        functions.getTableData2(output_file, url, browser)
    

In [19]:
retake_entity_df = pd.read_csv(output_file, encoding='latin1', names=col_names)

In [20]:
retake_entity_df.entity.value_counts()

Ministerio del Trabajo y Previsión Social            75041
b'Ministerio del Trabajo y Previsi\xf3n Social'      50531
b'Ministerio del Interior y Seguridad P\xfablica'    45599
Ministerio de Minería                                37565
b'Ministerio de Obras P\xfablicas'                   27766
b'Ministerio de Educaci\xf3n'                        19799
b'Ministerio de Econom\xeda, Fomento y Turismo'      14122
b'Ministerio de Justicia'                            12252
b'Ministerio de Desarrollo Social'                    9425
b'Ministerio de Defensa Nacional'                     4552
Ministerio de Energía                                 4545
Ministerio de Desarrollo Social                       4113
Ministerio de Educación                               3450
Ministerio de Justicia                                3168
Ministerio de Economía, Fomento y Turismo             3124
b'Ministerio de Salud'                                3089
Ministerio del Interior y Seguridad Pública           24

In [21]:
cleanLatin(retake_entity_df)

Could not clean column: other

Could not clean column: other

Could not clean column: other

Could not clean column: other

Could not clean column: other

Could not clean column: other

Could not clean column: other

Could not clean column: other

Could not clean column: other



### Let's delete the corresponding rows and add the new ones 

In [22]:
retake_entity_df['filename'] = output_file
print(retake_entity_df.shape)
print(dfscraped[dfscraped.entity.isin(retake_entity)].shape)
dfscraped = dfscraped[~dfscraped.entity.isin(retake_entity)]
dfscraped = pd.concat([dfscraped, retake_entity_df])
dfscraped.shape

(328844, 23)
(310284, 23)


(1917695, 23)

# Write clean file 

In [38]:
print(dfscraped.shape)
cleanLatin(dfscraped)
dfscraped1 = dfscraped.drop_duplicates()
print(dfscraped1.shape)
dfscraped1.to_csv('./output/scraped_data_clean.csv')

# Some stats 

In [30]:
# Be able to get first stats in Ministerio de Justicia

In [40]:
dfjusticia = dfscraped1.ix[dfscraped1.entity == 'Ministerio de Justicia']

In [41]:
dfjusticia.department.value_counts()

Servicio Nacional de Menores (SENAME)                  186277
Servicio de Registro Civil e Identificación (SRCEI)    149774
Subsecretaría de Justicia                                9613
Defensoría Penal Pública                                 6632
Name: department, dtype: int64

In [42]:
dfsename = dfjusticia.ix[dfjusticia.department == 'Servicio Nacional de Menores (SENAME)']

In [43]:
dfsename.year.value_counts()

Año 2015      12339
Año 2013       9832
SEPTIEMBRE     9353
OCTUBRE        9335
AGOSTO         9032
Año 2014       8873
DICIEMBRE      8853
JULIO          8794
NOVIEMBRE      8635
JUNIO          8606
MAYO           8532
ABRIL          8300
FEBRERO        8289
MARZO          8206
Año 2012       8124
Año 2011       8034
ENERO          7988
Año 2010       7970
Año 2008       7368
Año 2009       7180
Año 2007       7028
Año 2006       5512
Año 2016         94
Name: year, dtype: int64

In [37]:
dfsename.ix[dfsename.year == 'SEPTIEMBRE']

,entity,department,contract,year,month,group,lastn,lastn2,givenn,degree,...,specials,currency,salary,overt,start_date,end_date,obs,url,other,filename
714884,Ministerio de Justicia,Servicio Nacional de Menores (SENAME),Dotación de planta,SEPTIEMBRE,allyear,DIRECTIVO,LABRAÑA,SANTANA,MARCELA NOEMI,02,...,(2)(4)(9)(10),PESOS,4453583,No,28/03/2014,24/04/2016,RENUNCIA VOLUNTARIA,http://www.gobiernotransparentechile.cl/direct...,NaN,output\scraped_data_20170225.csv
714885,Ministerio de Justicia,Servicio Nacional de Menores (SENAME),Dotación de planta,SEPTIEMBRE,allyear,DIRECTIVO,HERRERA,ANDREUCCI,HUGO ANDRES,02,...,(4)(9)(10),PESOS,2872316,No,25/04/2016,15/07/2016,RENUNCIA VOLUNTARIA,http://www.gobiernotransparentechile.cl/direct...,NaN,output\scraped_data_20170225.csv
714886,Ministerio de Justicia,Servicio Nacional de Menores (SENAME),Dotación de planta,SEPTIEMBRE,allyear,DIRECTIVO,HUERTA,REYES,SOLANGE PAOLA,02,...,(4)(9)(10),PESOS,6416291,No,16/07/2016,Indefinido,NaN,http://www.gobiernotransparentechile.cl/direct...,NaN,output\scraped_data_20170225.csv
714887,Ministerio de Justicia,Servicio Nacional de Menores (SENAME),Dotación de planta,SEPTIEMBRE,allyear,DIRECTIVO,ASTUDILLO,ROJAS,JOHNNY ALEXIS,04,...,(1),PESOS,3339669,No,17/07/2015,Indefinido,NaN,http://www.gobiernotransparentechile.cl/direct...,NaN,output\scraped_data_20170225.csv
714888,Ministerio de Justicia,Servicio Nacional de Menores (SENAME),Dotación de planta,SEPTIEMBRE,allyear,DIRECTIVO,CANALES,RODRIGUEZ,JOSE MIGUEL,04,...,(2),PESOS,2840074,No,11/10/2014,07/08/2016,RENUNCIA A PLANTA Y PASA A CONTRATA,http://www.gobiernotransparentechile.cl/direct...,NaN,output\scraped_data_20170225.csv
714889,Ministerio de Justicia,Servicio Nacional de Menores (SENAME),Dotación de planta,SEPTIEMBRE,allyear,DIRECTIVO,CANALES,FIGUEROA,ARIEL NICOLAS,04,...,(5),PESOS,4567139,No,06/10/2015,01/03/2016,RENUNCIA AL SERVICIO,http://www.gobiernotransparentechile.cl/direct...,NaN,output\scraped_data_20170225.csv
714890,Ministerio de Justicia,Servicio Nacional de Menores (SENAME),Dotación de planta,SEPTIEMBRE,allyear,DIRECTIVO,CASTILLO,SILVA,CRISTIAN ANDRES,04,...,(2)(5),PESOS,4215591,No,01/02/2015,01/02/2018,"PERCIBE ASGINACION DE FUNCION CRITICA AL 25,85...",http://www.gobiernotransparentechile.cl/direct...,NaN,output\scraped_data_20170225.csv
714891,Ministerio de Justicia,Servicio Nacional de Menores (SENAME),Dotación de planta,SEPTIEMBRE,allyear,DIRECTIVO,DE FERARI,VIAL,LUIS IGNACIO,04,...,EN TRAMITE,PESOS,0,No,29/08/2016,31/12/2016,NaN,http://www.gobiernotransparentechile.cl/direct...,NaN,output\scraped_data_20170225.csv
714892,Ministerio de Justicia,Servicio Nacional de Menores (SENAME),Dotación de planta,SEPTIEMBRE,allyear,DIRECTIVO,LAVANDEROS,SVEC,JORGE ANTONIO,04,...,(2)(5),PESOS,4177930,No,21/12/2014,21/12/2017,"PERCIBE ASGINACION DE FUNCION CRITICA AL 23,50...",http://www.gobiernotransparentechile.cl/direct...,NaN,output\scraped_data_20170225.csv
714893,Ministerio de Justicia,Servicio Nacional de Menores (SENAME),Dotación de planta,SEPTIEMBRE,allyear,DIRECTIVO,MONTERO,BARRA,MARIA JOSE,05,...,(2)(5),PESOS,3561966,No,23/04/2014,Indefinido,"PERCIBE ASGINACION DE FUNCION CRITICA AL 18,70...",http://www.gobiernotransparentechile.cl/direct...,NaN,output\scraped_data_20170225.csv


In [44]:
dfsename.ix[714884, 'url']

'http://www.gobiernotransparentechile.cl/directorio/entidad/11/159/per_planta/SEPTIEMBRE'

In [45]:
dfsename.month.value_counts()

allyear    186277
Name: month, dtype: int64